In [1]:
# Import API modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import os
import geopandas as gpd
# Import linestrings
import tradeAreasPreprocessing as tap
from shapely.geometry import LineString
import census_finder

In [ ]:
# Read token.txt
with open("inrix_token.txt", "r") as f:
    token = f.read()

# Use API
endpoint = f"https://api.iq.inrix.com/v1/segments/speed?box=37.757386%7C-122.490667%2C37.746138%7C-122.395481"
r = requests.get(endpoint, headers={'Authorization': f"Bearer {token}"})

In [ ]:
od = 'origin'
geoFilterType = 'polygon'
radius = '0.2km'
points = '37.734622%7C-122.471603%2C37.743627%7C-122.463850%2C37.743066%7C-122.475429'
providerType = 'consumer'
startDateTime = '%3E%3D2023-06-01T02%3A31'
endDateTime = '%3C%3D2023-06-15T02%3A31'

# Trade Areas API
endpoint_TA = f'https://api.iq.inrix.com/v1/trips?od={od}&geoFilterType={geoFilterType}&radius={radius}&points={points}&providerType={providerType}&startDateTime={startDateTime}&endDateTime={endDateTime}'
r_TA = requests.get(endpoint_TA, headers={'Authorization': f"Bearer {token}"})

In [ ]:
# Convert dictionary to dataframe
df = pd.DataFrame.from_dict(r_TA.json()['data'])


In [ ]:
# Separate startLoc into two columns: all numbers before comma and all numbers after comma, then convert to floats
df['start_lat'] = df['startLoc'].apply(lambda x: x.split(',')[0]).astype(float)
df['start_lon'] = df['startLoc'].apply(lambda x: x.split(',')[1]).astype(float)
df['end_lat'] = df['endLoc'].apply(lambda x: x.split(',')[0]).astype(float)
df['end_lon'] = df['endLoc'].apply(lambda x: x.split(',')[1]).astype(float)

# Create linestring from start and end coordinates
df['geometry'] = df.apply(lambda x: LineString([(x['start_lon'], x['start_lat']), (x['end_lon'], x['end_lat'])]), axis=1)

In [ ]:
# Read geojson
sf = gpd.read_file('San_Francisco_Bay_Region_2020_Census_Tracts.geojson')

In [ ]:
# Convert df to geopandas
df = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

# Find intersection of linestring and geojson
sf['intersections'] = sf.geometry.clip(df.geometry)

In [ ]:
sf[sf['intersections'].notnull()].intersections

### Temporal Processing

In [ ]:
df['startDateTime'] = pd.to_datetime(df['startDateTime'], format='%Y-%m-%dT%H:%M:%S.%fZ')
df['endDateTime'] = pd.to_datetime(df['endDateTime'], format='%Y-%m-%dT%H:%M:%S.%fZ')

df['duration'] = (df['endDateTime'] - df['startDateTime']).dt.total_seconds()

df['day'] = df['startDateTime'].dt.day
df['month'] = df['startDateTime'].dt.month
df['year'] = df['startDateTime'].dt.year
df['dow'] = df['startDateTime'].dt.dayofweek
df['start_hour'] = df['startDateTime'].dt.hour
df['end_hour'] = df['endDateTime'].dt.hour
df['start_date'] = df['startDateTime'].dt.date
df['end_date'] = df['endDateTime'].dt.date
df['weekend'] = df['dow'].apply(lambda x: 1 if x >= 5 else 0)

## Weather API?

In [ ]:
with open("weather_token.txt", "r") as f:
    weather_token = f.read()

date = '2023-11-29'

city = 'San Francisco'
# Use API
endpoint_weather = f"http://api.weatherapi.com/v1/history.json?key={weather_token}&q={city}&dt={date}"
r_weather = requests.get(endpoint_weather)

r_weather.json()

In [ ]:
# Read as dataframe
df_weather = pd.DataFrame.from_dict(r_weather.json()['forecast']['forecastday'][0]['hour'])

In [ ]:
df_weather

In [10]:

# Read token.txt
with open("inrix_token.txt", "r") as f:
    token = f.read()
    
# Initialize class
tap = tap.tradeAreasPreprocessing(token=token,
                                  od = 'origin', 
                                  geoFilterType = 'polygon',
                                radius = '0.2km',
                                points = '37.734622%7C-122.471603%2C37.743627%7C-122.463850%2C37.743066%7C-122.475429',
                                providerType = 'consumer',
                                startDateTime = '%3E%3D2023-06-01T02%3A31',
                                endDateTime = '%3C%3D2023-06-15T02%3A31')

# Read data
df = tap.read_data()

AttributeError: 'tradeAreasPreprocessing' object has no attribute 'tradeAreasPreprocessing'

In [7]:
df = tap.create_dataframe(df)

KeyError: 'data'

In [4]:
df = tap.separate_coordinates(df)

df = tap.add_temporal_vars(df)

#df = tap.add_mode(df)

In [5]:
trips_gdf = tap.get_trip_geometry(df)

ca_tiger = tap.downloadShapefile()

Using FIPS code '06' for input 'CA'


In [6]:
intersected_trips = tap.intersectedTrips(trips_gdf, ca_tiger)

In [32]:
# Reload magic functions
%reload_ext autoreload
%autoreload 2

In [4]:
# read in files
geojson_location = "/Users/ekinokos2/Library/CloudStorage/OneDrive-UW/GeoAdVisor/sf-tracts-2020-clipped.geojson"
census_data = "/Users/ekinokos2/Library/CloudStorage/OneDrive-UW/GeoAdVisor/census_data/census_data_clean.csv"

# Load the GeoJSON file into a GeoDataFrame and csv into df
gdf = gpd.read_file(geojson_location)
census_df = pd.read_csv(census_data, dtype={"tract": str})

# Filter out invalid geometries
gdf = gdf[gdf.is_valid]
2
# INitialize class
cf = census_finder.censusFinder(gdf, census_df)

tract_no = cf.get_census_tract_info(37.734622, -122.471603)

demographics = cf.lookup_demographics(tract_no)

010101


In [22]:

# Example usage
latitude = 37.743138  # Replace with actual latitude
longitude = -122.466007  # Replace with actual longitude
census_tract_info = census_finder.get_census_tract_info(latitude, longitude, gdf)


In [23]:
census_tract_info

{'TRACTCE20': '030800'}

In [16]:
df.iloc[0]

providerHash          c3e878e27f52e2a57ace4d9a76fd9acf
startLoc                         37.743138,-122.466007
startDateTime               2023-06-01 02:35:57.079000
endLoc                           37.735084,-122.500487
endpointType                                         0
tripDistanceMeters                                3472
tripMeanSpeedKPH                                    35
tripId                dd8ed7439a33ebe0797e23cb7918ec21
endDateTime                 2023-06-01 02:41:56.818000
deviceId              c05484db3c0e831f242bdc0ce0c5e847
providerType                                  consumer
start_lat                                    37.743138
start_lon                                  -122.466007
end_lat                                      37.735084
end_lon                                    -122.500487
duration                                       359.739
day                                                  1
month                                                6
year      

In [14]:
demographics

,tract,Median Household Income,total_pop,pct_female,pct_male,pct_children,pct_20-39,pct_39-64,pct_65+,pct_nonwhite,pct_latine
170,030800,171250.0,6041.0,0.490316,0.509684,0.10048,0.023506,0.190697,0.098659,0.338851,0.049164


In [19]:
intersected_trips[intersected_trips['tripId'] == 'dd8ed7439a33ebe0797e23cb7918ec21']

,tripId,GEOID,geometry_x,geometry_y,line_locs
3658,dd8ed7439a33ebe0797e23cb7918ec21,06075030800,"POLYGON ((-122.47573 37.74305, -122.47558 37.7...","LINESTRING (-122.46601 37.74314, -122.50049 37...","(0.0, 1094.7372454378553)"
3659,dd8ed7439a33ebe0797e23cb7918ec21,06075033001,"POLYGON ((-122.49451 37.74222, -122.49436 37.7...","LINESTRING (-122.46601 37.74314, -122.50049 37...","(2878.403871091762, 3208.2974503973883)"
3660,dd8ed7439a33ebe0797e23cb7918ec21,06075033002,"POLYGON ((-122.49129 37.73730, -122.49090 37.7...","LINESTRING (-122.46601 37.74314, -122.50049 37...","(1094.7372454378553, 2878.403871091762)"
3661,dd8ed7439a33ebe0797e23cb7918ec21,06075060400,"POLYGON ((-122.51273 37.73509, -122.51008 37.7...","LINESTRING (-122.46601 37.74314, -122.50049 37...","(3931.2131256589555, 3934.2353148580683)"
3662,dd8ed7439a33ebe0797e23cb7918ec21,06075035300,"POLYGON ((-122.50193 37.74938, -122.50086 37.7...","LINESTRING (-122.46601 37.74314, -122.50049 37...","(3208.2974503973883, 3931.2131256589555)"
